## Grundlagen Maschineller Lernverfahren | ML_INF19A | 2021
**Datum: 05.10.2021**

# Optimierung

In [ ]:
# Lade notwendige Bibliotheken

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Pseudo Zufallszahlen

np.random.seed(2020)

## Daten erstellen

In [ ]:
# Daten erstellen 

X = 2*np.random.rand(100,1)
y = 4+3*X + np.random.rand(100,1)

In [ ]:
# Daten anzeigen

plt.figure(figsize=(5,3))
plt.plot(X,y,'b.')
plt.xlabel("x")
plt.ylabel("y")
plt.show()

In [ ]:
# Erstelle zwei x-Werte zum Testen

X_unknown = np.array([[0],[2]])

## SGD Modell erstellen

Ressourcen: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html

In [ ]:
# Lade SGD Modell

from sklearn.linear_model import SGDRegressor

# Erstelle SGDRegressor Instanz
sgd_reg = SGDRegressor() # Standardwerte! -> Verweis auf Dokumentation

# Starte Training
sgd_reg.fit(X,y.ravel())

# Hinweis: Die Dimension des Arrays ist sehr oft kritisch und muss entsprechend angepasst werden.
# Oft hilft hier die Numpy Funktion "ravel"  weiter: https://numpy.org/doc/stable/reference/generated/numpy.ravel.html

# Erhalte Vorhersage
y_pred = sgd_reg.predict(X_unknown)

# Zeige Vorhersage
print(y_pred)

In [ ]:
# Analysiere LR Ergebnis -> w0 und w1

b = sgd_reg.intercept_ # w0
w = sgd_reg.coef_ # w1

print(b,w)

Erinnerung (Geradengleichung):

$h_w(x) = b + m*x = w_0 + w_1*x_1$

# Vergleich mit Lin Reg

Ressource: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

In [ ]:
# Lade LinReg Modell

from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression() # Standardwerte! -> Verweis auf Dokumentation
lin_reg.fit(X,y)

y_pred_lr = lin_reg.predict(X_unknown)

# Analysiere LR Ergebnis -> w0 und w1

b_lr = lin_reg.intercept_ # w0
w_lr = lin_reg.coef_ # w1

print(b_lr,w_lr)

# Visualisierung

In [ ]:
# Für Anzeige der Geraden

Xline = np.linspace(0,2,100)

In [ ]:
# Plot der Datenmenge und der Gerade und der beiden neuen X_unknown

plt.figure(figsize=(15,10))

# Datenmenge
plt.plot(X,y,'b.')

# SGD
plt.plot(X_unknown[0,0],y_pred[0],'r^')
plt.plot(X_unknown[1,0],y_pred[1],'r^')

plt.plot(Xline, w[0]*Xline + b,'r')

# LR
plt.plot(X_unknown[0,0],y_pred_lr[0],'m^')
plt.plot(X_unknown[1,0],y_pred_lr[1],'m^')

plt.plot(Xline, w_lr[0]*Xline + b_lr,'m')

# Sonstiges
plt.xlabel("x")
plt.ylabel("y")

plt.show()

# Hyperparameter der SGD ändern

Es werden unterschiedlich parametrierte SGDRegressor Modelle erstellt.


In [ ]:
# Instanzen anlegen
sgd_reg1 = SGDRegressor(loss="huber") # Standard = "squared_loss"
sgd_reg2 = SGDRegressor(max_iter=500) # Standard = 1000
sgd_reg3 = SGDRegressor(learning_rate="constant", eta0=0.000001) # Standard = "invscaling" / 0.01
sgd_reg4 = SGDRegressor(learning_rate="constant", eta0=0.05)
sgd_reg5 = SGDRegressor(loss="huber", max_iter=500) 
sgd_reg6 = SGDRegressor(learning_rate="constant", eta0=0.05, max_iter=2000) 

# Instanzen zusammenfassen (für Schleife)
sgd_list = [sgd_reg, sgd_reg1, sgd_reg2, sgd_reg3, sgd_reg4, sgd_reg5, sgd_reg6]
colorlist = ['b','g','r','c','m','y','k']

plt.figure(figsize=(15,10))

# Datenmenge
plt.plot(X,y,'b.')

c = -1 # Counter für Farben
for sgd in sgd_list:
  c += 1 # Liste startet mit 0
  
  # Training
  sgd.fit(X,y.ravel())
 
  # Vorhersage
  y_pred_i = sgd.predict(X_unknown)

  # Gewichte auslesen
  b = sgd.intercept_ # w0
  w = sgd.coef_ # w1

  # Gebe relevante Informationen des aktuellen Modells aus
  print("[SGD Hyperparams] %s\t%d\t%s (%f)" % (sgd.loss, sgd.max_iter, sgd.learning_rate, sgd.eta0))
  print("b=%f\tw=%f\ty_pred=%f\n\n" % (b[0],w[0],y_pred_i[0]))

  # Zeichne aktuelles Modell
  plt.plot(X_unknown[0,0],y_pred_i[0],"%s^" % (colorlist[c]))
  plt.plot(X_unknown[1,0],y_pred_i[1],"%s^" % (colorlist[c]))
  plt.plot(Xline, w[0]*Xline + b,colorlist[c],label="Modell sgd_reg%d" % (c))

  plt.xlabel("x")
  plt.ylabel("y")

  plt.legend()

# Zeige Plot mit allen Modellen
plt.show()

# Selbststudium

Im Folgenden wird kurz gezeigt wie die Eingangsdaten $\vec x$ des Modells "skaliert" werden können.
Für viele Modelle ist es zwingend notwendig die Eingangsdatensätze in einen gewissen Wertebereich zu transformieren: oft sollten die Eingangsdaten so aussehen, als ob diese "normalverteilt" sind.

Im Folgenden wird die Anwendung des "StandardScaler" vorgestellt, der die Daten entsprechend "verteilt".

Weitere Informationen dazu gibt es hier: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html


In [ ]:
# Lade Bibliothek
from sklearn.preprocessing import StandardScaler

# Erstelle Instanz
scaler = StandardScaler()

# "Lerne" die Instanz mit den zur Verfügung stehenden Daten an (= Analysiere)
# Hinweis: Es werden immer nur die Eingangsdaten $x$ skaliert, *nicht* die Zielvariable!
scaler.fit(X)

# Wo liegt der Erwartungswert der Daten X?
print("[StandardScaler] Mean = %f " % (scaler.mean_))

# Führe die Skalierung durch 1) Trainingsdaten & 2) Anwendungsdaten (unbekannte Daten)
X_scaled = scaler.transform(X)
X_unknown_scaled = scaler.transform(X_unknown) # Transformation gemäß den Trainingsdaten


## Visualisiere

In [ ]:
# Zeichne die Datenpunkte
plt.figure(figsize=(15,10))

# Datenmenge
plt.plot(X,y,'b.') # ursprüngliche Datenmenge
plt.plot(X_scaled,y,'r.') # transformierte Datenmenge

plt.xlabel("x")
plt.ylabel("y")

plt.show()

## Anwendung mit Modell

In [ ]:
# Erstelle eine neue Modellinstanz
sgd_reg_scaling = SGDRegressor() # Standardwerte! -> Verweis auf Dokumentation

# Starte Training
sgd_reg_scaling.fit(X_scaled,y.ravel())

# Erhalte Vorhersage
y_pred_scaled = sgd_reg_scaling.predict(X_unknown_scaled)

# Zeige Vorhersage
print(y_pred_scaled)

In [ ]:
# Vgl. Vorhersage mit nicht transformierten Daten
print(y_pred)